## Accuracy of CasCADe

This section calculates the results presented in Table 2 in Section 5

### Some Utility Functions

In [1]:
import csv

In [2]:
def get_ccc_gt():
    cve_casualties = {}
    file_name = "SecurityDataset/ground_truth.csv"
    with open(file_name, 'r', encoding='latin-1') as file:
        csv_reader = csv.reader(file, delimiter="\t")
        count = 0
        for row in csv_reader:
                if row[0] and '1' in row: 
                    count += 1

                    #the cve id is in the fifth cell
                    cve_name = row[0].strip()

                    #the method name is in the sixth cell
                    method_name = row[1].strip().replace("\"", '')

                    if cve_name in cve_casualties.keys():
                        if method_name not in cve_casualties[cve_name]:
                            cve_casualties[cve_name].append(method_name)
                    else:
                        cve_casualties[cve_name] = [method_name]
    return cve_casualties

In [3]:
def get_cco_gt():
    cve_casualties = {}
    file_name = "SecurityDataset/ground_truth.csv"
    with open(file_name, 'r', encoding='latin-1') as file:
        csv_reader = csv.reader(file, delimiter="\t")
        count = 0
        for row in csv_reader:
                if row[0] and '1' in row and ('4' in row or '3' in row or '2' in row): 
                    count += 1

                    #the cve id is in the fifth cell
                    cve_name = row[0].strip()

                    #the method name is in the sixth cell
                    method_name = row[1].strip().replace("\"", '')

                    if cve_name in cve_casualties.keys():
                        if method_name not in cve_casualties[cve_name]:
                            cve_casualties[cve_name].append(method_name)
                    else:
                        cve_casualties[cve_name] = [method_name]
    return cve_casualties

In [4]:
#function to get the ground truth from our ground truth dataset 
#takes as parameter an integer which represents the code for the CC 
# 2 - refactoring, 3 - api changes, 4 getter and setter 

def get_gt(type, *args):

    cve_casualties = {}
    file_name = "SecurityDataset/ground_truth.csv"
    with open(file_name, 'r', encoding='latin-1') as file:
        csv_reader = csv.reader(file, delimiter="\t")
        count = 0
        for row in csv_reader:
                if not args: 
                    if row[0] and '1' in row and str(type) in row:
                        count += 1

                        #the cve id is in the fifth cell
                        cve_name = row[0].strip()

                        #the method name is in the sixth cell
                        method_name = row[1].strip().replace("\"", '')

                        if cve_name in cve_casualties.keys():
                            if method_name not in cve_casualties[cve_name]:
                                cve_casualties[cve_name].append(method_name)
                        else:
                            cve_casualties[cve_name] = [method_name]
                else: 
                    arg = args[0]
                    condition = row[0] and '1' in row and (str(type) in row or str(arg) in row)
                    if len(args)==2:
                        
                        condition = row[0] and '1' in row and (str(type) in row or str(arg) in row or str(args[1]) in row)
     
                    if condition:
                        count += 1

                        #the cve id is in the fifth cell
                        cve_name = row[0].strip()

                        #the method name is in the sixth cell
                        method_name = row[1].strip().replace("\"", '')

                        if cve_name in cve_casualties.keys():
                            if method_name not in cve_casualties[cve_name]:
                                cve_casualties[cve_name].append(method_name)
                        else:
                            cve_casualties[cve_name] = [method_name]

    return cve_casualties

In [5]:
# some methods outputted have preceeding and trailing "<", ">", respectively. We remove them 

def clean_methods(elems):
    result = []
    for elem in elems: 
        result.append(elem.replace("<", '').replace(">", '').split("(")[0].strip())
    return result

In [6]:
#obtain cascade output 
def get_cascade_output(file_name):

    automated_casualties = {}

    with open(file_name, 'r') as file:
        for line in file:
            line = line.strip()
            line = line.split(";")
            if len(line)>2:
                cve = line[0].strip()
                if cve in automated_casualties.keys():
                    for index in range(1, len(line)):

                        elem = line[index].strip().split("#")[0]

                        if not elem.isdigit() and elem not in automated_casualties[cve]:
                            automated_casualties[cve].append(elem)
                else:
                    elems = []
                    for index in range(1, len(line)):
                        elem = line[index].strip().split("#")[0]

                        if not elem.isdigit() and elem not in elems:
                            elems.append(elem)
                    automated_casualties[cve] = elems
    return automated_casualties

In [7]:
def add_to_dict(file_name, automated_casualties):
        with open(file_name, 'r') as file:
            for line in file:
                line = line.strip()
                line = list(filter(None, line.split(";")))
                if len(line)>1:
                    cve = line[0].strip()
                    if cve in automated_casualties.keys():
                        for index in range(1, len(line)):

                            elem = line[index].strip().split("#")[0]

                            if len(elem)>0 and elem != '\n' and not elem.isdigit() and elem not in automated_casualties[cve]:
                                automated_casualties[cve].append(elem)
                    else:
                        elems = []
                        for index in range(1, len(line)):
                            elem = line[index].strip().split("#")[0]

                            if len(elem)>0 and elem != '\n' and not elem.isdigit() and elem not in elems:
                                elems.append(elem)
                        automated_casualties[cve] = elems

In [8]:
def count_sec_methods(file_name):
    count = 0;
    with open(file_name, "r") as fi:
        for line in fi: 
            count += 1
    return count

In [9]:
def count_methods(file_name):
    count = 0;
    with open(file_name, "r") as fi:
        for line in fi: 
            line_spl = list(filter(None, line.strip().split(";")))
            if len(line_spl)>1:
                count += len(line_spl) - 1
    return count

In [10]:
def count_methods_zxing(file_name1, file_name2):
    count = 0;
    with open(file_name1, "r") as fi:
        for line in fi: 
            line_spl = list(filter(None, line.strip().split(";")))
            if len(line_spl)>1:
                count += len(line_spl) - 1
    with open(file_name2, "r") as fi:
        for line in fi: 
            line_spl = list(filter(None, line.strip().split(";")))
            if len(line_spl)>1:
                count += len(line_spl) - 1
    return count

In [11]:
def count_cc_methods(casualty_dict):
    count = 0;
    for key, elem in casualty_dict.items():
        
        count += len(elem)
    return count

In [12]:
#get output for zxing 
def get_cascade_combined_output_zxing(file_name1, file_name2, file_name3, file_name4, file_name5, file_name6):

    automated_casualties = {}

    add_to_dict(file_name1, automated_casualties)
    add_to_dict(file_name2, automated_casualties)
    add_to_dict(file_name3, automated_casualties)
    add_to_dict(file_name4, automated_casualties)
    add_to_dict(file_name5, automated_casualties)
    add_to_dict(file_name6, automated_casualties)
       
    return automated_casualties

In [13]:
#obtain combined cascade output 
def get_cascade_combined_output(file_name1, file_name2, file_name3):

    automated_casualties = {}

    add_to_dict(file_name1, automated_casualties)
    add_to_dict(file_name2, automated_casualties)
    add_to_dict(file_name3, automated_casualties)
    return automated_casualties

In [14]:
def get_precision(automated_casualties, gt_casualties):
    tp = 0
    total = 0
    for key, elem in automated_casualties.items():
        elem = clean_methods(elem)
        if key in gt_casualties.keys():
            elems = clean_methods(gt_casualties[key])
            for el in elem: 
                if(len(el)>0):
                    if el in elems:
                        tp+= 1
                    total += 1
        else:
            for el in elem:
                if len(el)>0:
                    total += 1
    return tp, total

In [15]:
def get_recall(gt_casualties, automated_casualties):
    tp = 0
    total = 0
    for key, elem in gt_casualties.items():
        elem = clean_methods(elem)
        if key in automated_casualties.keys():
            elems = clean_methods(automated_casualties[key])
            for el in elem: 
                if(len(el)>0):
                    if el in elems:
                        tp+= 1
                    total += 1
        else:
            for el in elem:
                if len(el)>0:
                    total += 1
    return tp, total

### API-based casualties

#### CC-O

In [16]:
#get the ground truth 
gt_cco_api = get_gt(3)

In [17]:
#get the cascade results
api_cco_file_name = "SecurityDataset/output/apiCasCO.out"
automated_cco_api = get_cascade_output(api_cco_file_name)

In [18]:
#calculate metrics

In [19]:
tp_p_cco_api, total_p_cco_api = get_precision(automated_cco_api, gt_cco_api)

In [20]:
precision = tp_p_cco_api/total_p_cco_api

In [21]:
precision

1.0

In [22]:
tp_r_cco_api, total_r_cco_api = get_recall(gt_cco_api, automated_cco_api)

In [23]:
recall = tp_r_cco_api/total_r_cco_api

In [24]:
recall

1.0

In [25]:
total_api_cas = count_cc_methods(gt_cco_api)

In [26]:
total_api_cas

85

#### CC-C

In [27]:
#get ground truth results
gt_ccc_api = get_gt(3, 3.1)

In [28]:
#get cascasde results 
api_ccc_file_name = "SecurityDataset/output/apiCasCC.out"
automated_ccc_api = get_cascade_output(api_ccc_file_name)

In [29]:
#calculate metrics

In [30]:
tp_p_ccc_api, total_p_ccc_api = get_precision(automated_ccc_api, gt_ccc_api)

In [31]:
precision = tp_p_ccc_api/total_p_ccc_api

In [32]:
precision

0.96

In [33]:
tp_r_ccc_api, total_r_ccc_api = get_recall(gt_ccc_api, automated_ccc_api)

In [34]:
recall = tp_r_ccc_api/total_r_ccc_api

In [35]:
recall

0.8962264150943396

In [36]:
total_api_ccc = count_cc_methods(gt_ccc_api)

In [37]:
total_api_ccc

106

### Variable-based casualties

#### CC-O

In [38]:
#get ground truth results 
gt_cco_mvar = get_gt(4) 

In [39]:
#get cascade results
var_cco_file_name = "SecurityDataset/output/mvarCasCO.out"
automated_cco_mvar = get_cascade_output(var_cco_file_name)

In [40]:
#calculate metrics 

In [41]:
tp_p_mvar, total_p_mvar = get_precision(automated_cco_mvar, gt_cco_mvar)

In [42]:
precision = tp_p_mvar/total_p_mvar

In [43]:
precision

0.8789473684210526

In [44]:
tp_r_mvar, total_r_mvar = get_recall(gt_cco_mvar, automated_cco_mvar)

In [45]:
recall = tp_r_mvar/total_r_mvar

In [46]:
recall

0.8542713567839196

In [47]:
total_var_ccc = count_cc_methods(gt_cco_mvar)
total_var_ccc

199

#### CC-C

In [48]:
#get the ground truth results 
gt_ccc_mvar=get_gt(4, 4.1)

In [49]:
#get the cascade output
var_ccc_file_name = "SecurityDataset/output/mvarCasCC.out"
automated_ccc_mvar = get_cascade_output(var_ccc_file_name)

In [50]:
#calculate the results

In [51]:
tp_p_ccc_mvar, total_p_ccc_mvar = get_precision(automated_ccc_mvar, gt_ccc_mvar)

In [52]:
precision = tp_p_ccc_mvar/total_p_ccc_mvar

In [53]:
precision

0.7809187279151943

In [54]:
tp_r_ccc_mvar, total_r_ccc_mvar = get_recall(gt_ccc_mvar, automated_ccc_mvar)

In [55]:
recall = tp_r_ccc_mvar/total_r_ccc_mvar

In [56]:
recall

0.9306122448979591

In [57]:
total_var_ccc = count_cc_methods(gt_ccc_mvar)
total_var_ccc

245

### Ref-based casualties 

#### CC-O

In [58]:
#get the ground truth results 
gt_cco_ref = get_gt(2)

In [59]:
#get the cascade output 
ref_cco_filename = "SecurityDataset/output/refCasCO.out"
automated_cco_ref = get_cascade_output(ref_cco_filename)

In [60]:
#calculate the metrics

In [61]:
tp_p_cco_ref, total_p_cco_ref = get_precision(automated_cco_ref, gt_cco_ref)

In [62]:
precision = tp_p_cco_ref/total_p_cco_ref

In [63]:
precision

1.0

In [64]:
tp_r_cco_ref, total_r_cco_ref = get_recall(gt_cco_ref, automated_cco_ref)

In [65]:
recall = tp_r_cco_ref/total_r_cco_ref

In [66]:
recall

1.0

In [67]:
total_ref_cco = count_cc_methods(gt_cco_ref)

In [68]:
total_ref_cco

29

### CC-C

In [69]:
#get the ground truth results 
gt_ccc_ref = get_gt(2, 2.1)

In [70]:
#get the cascade output 
ref_ccc_filename = "SecurityDataset/output/refCasCC.out"
automated_ccc_ref = get_cascade_output(ref_ccc_filename)

In [71]:
#calculate the metrics

In [72]:
tp_p_ccc_ref, total_p_ccc_ref = get_precision(automated_ccc_ref, gt_ccc_ref)

In [73]:
precision = tp_p_ccc_ref/total_p_ccc_ref

In [74]:
precision

0.975

In [75]:
tp_r_ccc_ref, total_r_ccc_ref = get_recall(gt_ccc_ref, automated_ccc_ref)

In [76]:
recall = tp_r_ccc_ref/total_r_ccc_ref

In [77]:
recall

1.0

In [78]:
total_ref_ccc = count_cc_methods(gt_ccc_ref)

In [79]:
total_ref_ccc

39

## Prevalence of Casualty Changes

This section presents the data used Table 1, Section 5.2 and in Section 5.3 (Figure 13) 

### Security Dataset

#### Patches

In [80]:
#all casualties
cve_ccc = get_ccc_gt()

In [81]:
#total number of patches
sec_total = 358

In [82]:
ccc_patches = list(cve_ccc.keys())

In [83]:
len(ccc_patches)/sec_total

0.2430167597765363

In [84]:
cve_cco = get_cco_gt()

In [85]:
cco_patches = list(cve_cco.keys())

In [86]:
len(cco_patches)/sec_total

0.20949720670391062

#### Methods

In [87]:
all_methods_fn = "SecurityDataset/ground_truth.csv"
sec_all_methods = count_sec_methods(all_methods_fn)

In [88]:
#used as total in table 1
sec_ccc_methods = count_cc_methods(cve_ccc)

In [89]:

sec_ccc_methods/sec_all_methods

0.23309178743961353

In [90]:
#used as total in table 1 
sec_cco_methods = count_cc_methods(cve_cco)

In [91]:
sec_cco_methods/sec_all_methods

0.1890096618357488

### General Dataset

#### Patches

In [92]:
ant_total = 3832
xerces_total = 2597
tomcat_total = 3163
zxing_total= 231
aspectj_total = 504

In [93]:
ant_ccc = get_cascade_combined_output("GeneralDataset/ant/output/apiCasCC.out", "GeneralDataset/ant/output/mvarCasCC.out", "GeneralDataset/ant/output/refCasCC.out")

In [94]:
ant_cco = get_cascade_combined_output("GeneralDataset/ant/output/apiCasCO.out", "GeneralDataset/ant/output/mvarCasCO.out", "GeneralDataset/ant/output/refCasCO.out")

In [95]:
xerces_ccc = get_cascade_combined_output("GeneralDataset/xerces/output/apiCasCC.out", "GeneralDataset/xerces/output/mvarCasCC.out", "GeneralDataset/xerces/output/refCasCC.out")

In [96]:
xerces_cco = get_cascade_combined_output("GeneralDataset/xerces/output/apiCasCO.out", "GeneralDataset/xerces/output/mvarCasCO.out", "GeneralDataset/xerces/output/refCasCO.out")

In [97]:
tomcat_ccc = get_cascade_combined_output("GeneralDataset/tomcat/output/apiCasCC.out", "GeneralDataset/tomcat/output/mvarCasCC.out", "GeneralDataset/tomcat/output/refCasCC.out")

In [98]:
tomcat_cco = get_cascade_combined_output("GeneralDataset/tomcat/output/apiCasCO.out", "GeneralDataset/tomcat/output/mvarCasCO.out", "GeneralDataset/tomcat/output/refCasCO.out")

In [99]:
zxing_ccc = get_cascade_combined_output_zxing("GeneralDataset/zxing/output/apiCasCC_1.out", "GeneralDataset/zxing/output/apiCasCC_2.out", "GeneralDataset/zxing/output/mvarCasCC_1.out", "GeneralDataset/zxing/output/mvarCasCC_2.out", "GeneralDataset/zxing/output/refCasCC_1.out", "GeneralDataset/zxing/output/refCasCC_2.out")

In [100]:
zxing_cco = get_cascade_combined_output_zxing("GeneralDataset/zxing/output/apiCasCO_1.out", "GeneralDataset/zxing/output/apiCasCO_2.out", "GeneralDataset/zxing/output/mvarCasCO_1.out", "GeneralDataset/zxing/output/mvarCasCO_2.out", "GeneralDataset/zxing/output/refCasCO_1.out", "GeneralDataset/zxing/output/refCasCO_2.out")

In [101]:
aspectj_ccc = get_cascade_combined_output("GeneralDataset/aspectj/output/apiCasCC.out", "GeneralDataset/aspectj/output/mvarCasCC.out", "GeneralDataset/aspectj/output/refCasCC.out")

In [102]:
aspectj_cco = get_cascade_combined_output("GeneralDataset/aspectj/output/apiCasCO.out", "GeneralDataset/aspectj/output/mvarCasCO.out", "GeneralDataset/aspectj/output/refCasCO.out")

In [103]:
gen_total_patches = ant_total + xerces_total + tomcat_total + zxing_total + aspectj_total

In [104]:
gen_cc_patches = len(ant_ccc) + len(xerces_ccc) + len(tomcat_ccc) + len(zxing_ccc) + len(aspectj_ccc)

In [105]:
gen_co_patches = len(ant_cco) + len(xerces_cco) + len(tomcat_cco) + len(zxing_cco) + len(aspectj_cco)

In [106]:
gen_cc_patches/gen_total_patches

0.12481843710661374

In [107]:
gen_co_patches/gen_total_patches

0.1089377360317614

#### Methods

In [108]:
def print_dict(to_print):
    for key, elem in to_print.items():
        print(f"{key}: {elem}")

In [109]:
ant_all_methods = count_methods("GeneralDataset/ant/output/allChangedMethods.out")
ant_cco_methods = count_cc_methods(ant_cco)
ant_ccc_methods = count_cc_methods(ant_ccc)

In [110]:
xerces_all_methods = count_methods("GeneralDataset/xerces/output/allChangedMethods.out")
xerces_cco_methods = count_cc_methods(xerces_cco)
xerces_ccc_methods = count_cc_methods(xerces_ccc)

In [111]:
tomcat_all_methods = count_methods("GeneralDataset/tomcat/output/allChangedMethods.out")
tomcat_cco_methods = count_cc_methods(tomcat_cco)
tomcat_ccc_methods = count_cc_methods(tomcat_ccc)

In [112]:
zxing_all_methods = count_methods_zxing("GeneralDataset/zxing/output/allChangedMethods_1.out", "GeneralDataset/zxing/output/allChangedMethods_2.out")
zxing_cco_methods = count_cc_methods(zxing_cco)
zxing_ccc_methods = count_cc_methods(zxing_ccc)

In [113]:
aspectj_all_methods = count_methods("GeneralDataset/aspectj/output/allChangedMethods.out")
aspectj_cco_methods = count_cc_methods(aspectj_cco)
aspectj_ccc_methods = count_cc_methods(aspectj_ccc)

In [114]:
gen_all_methods = ant_all_methods + xerces_all_methods + tomcat_all_methods + zxing_all_methods + aspectj_all_methods

In [115]:
gen_cco_methods = ant_cco_methods + xerces_cco_methods + tomcat_cco_methods + zxing_cco_methods + aspectj_cco_methods

In [116]:
gen_cco_methods_perc = 100*(gen_cco_methods/gen_all_methods)

In [117]:
gen_ccc_methods = ant_ccc_methods + xerces_ccc_methods + tomcat_ccc_methods + zxing_ccc_methods + aspectj_ccc_methods

In [118]:
gen_ccc_methods_perc = 100*(gen_ccc_methods/gen_all_methods)

In [119]:
gen_ccc_methods_perc

17.10912591937877

In [120]:
gen_cco_methods_perc

12.304681334638088